In [ ]:
import sympy as sp
from sympy.tensor.array import * 
import numpy as np 
from ipywidgets import interact, interact_manual,fixed
from ipywidgets import widgets # Widget definitions
from IPython.display import display, HTML, Math, Latex # Used to display widgets in the notebook
from sympy.utilities.codegen import codegen
from sympy.interactive import printing
printing.init_printing(use_latex=True)
import re

import timeit 

In [ ]:
t, x, y, z = sp.symbols('t,x,y,z')
psi=sp.Function('psi')(t,x,y,z)
phi=sp.Function('phi')(t,x,y,z)
alpha_GB=sp.symbols('alpha_GB')


In [ ]:
Dpsi=Array([psi.diff(c) for c in [x,y,z]])
Dphi=Array([phi.diff(c) for c in [x,y,z]])

In [ ]:
DDpsi= Array([[psi.diff(c,d) for c in [x,y,z]] for d in [x,y,z]])
DDphi= Array([[phi.diff(c,d) for c in [x,y,z]] for d in [x,y,z]])

In [ ]:
h=Array(sp.eye(3))
ih=Array(sp.eye(3))

In [ ]:
def print_olliptic_code(expr):
    print('input:')
    display(expr)
    expr_str=str( expr)
    expr_str=expr_str.replace('(t, x, y, z)','')
    expr_str=re.sub(r'Derivative\((\w*),\s([x,y,z]?)\)',r'\1_d\2',expr_str)
    expr_str=re.sub(r'Derivative\((\w*),\s([x,y,z]?),\s([x,y,z]?)\)',r'\1_d\2d\3',expr_str)
    print('output:')
    sp.printing.print_ccode( expr_str )



In [ ]:
def term0(a,b):
    return ih[a,b]*Dphi[a]*Dphi[b]

t1=0
for a in [0,1,2]:
    for b in [0,1,2]:
        t1+=term0(a,b)
t1=-psi**4*t1/4    
t1

In [ ]:
my_expr=t1.factor(psi**4)
print_olliptic_code(my_expr)
my_expr

In [ ]:
def term1(a,b,c,d):
    return ih[a,c]*ih[b,d]*Dphi[a]*Dphi[b]*Dpsi[c]*Dpsi[d]

def term2(a,b,c,d):
    return ih[a,b]*ih[c,d]*Dphi[a]*Dphi[b]*Dpsi[c]*Dpsi[d]

def term3(a,b,c,d):
    return ih[a,b]*ih[c,d]*Dphi[a]*Dpsi[b]*Dpsi[c]*Dpsi[d]

def term4(a,b,c,d):
    return ih[a,b]*ih[c,d]*DDphi[c,a]*DDpsi[d,b]

def term5(a,b,c,d):
    return ih[a,c]*ih[b,d]*Dpsi[a]*Dpsi[b]*DDpsi[d,c]

def term6(a,b,c,d):
    return ih[a,b]*ih[c,d]*Dpsi[a]*Dpsi[b]*DDphi[d,c]

def term7(a,b,c,d):
    return ih[a,c]*ih[b,d]*Dphi[a]*Dphi[b]*DDpsi[d,c]

def term8(a,b,c,d):
    return ih[a,b]*ih[c,d]*Dphi[a]*Dphi[b]*DDpsi[d,c]

def term9(a,b,c,d):
    return ih[a,c]*ih[b,d]*Dphi[a]*Dpsi[b]*DDpsi[d,c]

def term10(a,b,c,d):
    return ih[a,b]*ih[c,d]*DDphi[b,a]*DDpsi[d,c]



t2=0
for a in [0,1,2]:
    for b in [0,1,2]:
        for c in [0,1,2]:
            for d in [0,1,2]:
                t2+= 24*term1(a,b,c,d)/psi**2 - 8*term2(a,b,c,d)/psi**2
                t2+= 64*term3(a,b,c,d)/psi**3 + 8*term4(a,b,c,d)/psi
                t2+= 24*term5(a,b,c,d)/psi**2 + 8*term6(a,b,c,d)/psi**2
                t2+=  8*term7(a,b,c,d)/psi    + 8*term8(a,b,c,d)/psi
                t2+= 32*term9(a,b,c,d)/psi**2 - 8*term10(a,b,c,d)/psi

                
t2=alpha_GB*sp.exp(-phi)*t2
rho=t1+t2

In [ ]:
print_olliptic_code((-rho/(4*psi**3)).expand())

In [ ]:
def term3(a,b,c,d):
    return ih[a,c]*ih[b,d]*Dphi[a]*Dphi[b]*Dpsi[c]*Dpsi[d]

t2=0
for a in [0,1,2]:
    for b in [0,1,2]:
        for c in [0,1,2]:
            for d in [0,1,2]:
                t2+=term2(a,b,c,d)
                
t2=24*alpha_GB*sp.exp(-phi)*t2/psi**2    
t2

In [ ]:
re.sub(r'Derivative\((\w*),\s([x,y,z]?)\)',r'\1_d\2','pow(Derivative(phi, x), 2)')

In [ ]:
regex = re.compile(r'Derivative\(\w*,\s[x,y,z]?\)')

In [ ]:
regex.sub(r'\0','Derivative(phi, x)')

In [ ]:
help(re.sub)

In [ ]:
DDpsi[1,2]

In [ ]:
DDpsi[2,1]

In [ ]:
t2.expand().factor(Dpsi.tolist())

In [ ]:
sp.exp(-(x-y)**2)